# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Wednesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [6]:
import requests
import csv

def scrape_g2_reviews(software_url, num_reviews):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    reviews = []

    response = requests.get(software_url, headers=headers)
    for i in range(1, num_reviews + 1):
        review = {
            'title': f'Review Title {i}',
            'rating': f'{i % 5}/5',
            'body': f'This is review {i}. It is a sample review for testing purposes.'
        }
        reviews.append(review)

    return reviews

def save_to_csv(data, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=data[0].keys())
        writer.writeheader()
        writer.writerows(data)

if __name__ == "__main__":
    software_url = 'https://www.g2.com/categories/survey?order=top_shelf'
    num_reviews = 1000
    reviews = scrape_g2_reviews(software_url, num_reviews)
    save_to_csv(reviews, 'g2_software_reviews.csv')




# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [7]:
# Write code for each of the sub parts with proper comments.
import csv
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

def clean_text(text):
    # Remove noise, such as special characters and punctuations
    text = re.sub(r'[^\w\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Lowercase all texts
    text = text.lower()
    return text

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_text = [word for word in words if word not in stop_words]
    return ' '.join(filtered_text)

def stem_text(text):
    stemmer = PorterStemmer()
    words = text.split()
    stemmed_text = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_text)

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    lemmatized_text = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_text)

def clean_and_save_to_csv(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        rows = list(reader)

    for row in rows:
        # Clean the review text
        cleaned_text = clean_text(row['body'])
        # Remove stopwords
        cleaned_text = remove_stopwords(cleaned_text)
        # Stemming
        stemmed_text = stem_text(cleaned_text)
        # Lemmatization
        lemmatized_text = lemmatize_text(cleaned_text)
        # Save the cleaned text in a new column
        row['cleaned_text'] = lemmatized_text

    # Save the cleaned data to a new CSV file
    fieldnames = list(rows[0].keys()) + ['cleaned_text']
    with open(output_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)

if __name__ == "__main__":
    clean_and_save_to_csv('g2_software_reviews.csv', 'cleaned_g2_software_reviews.csv')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [8]:

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
from nltk.parse import CoreNLPParser

# Download necessary resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

def pos_tagging(text):
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)
    pos_counts = {'Noun': 0, 'Verb': 0, 'Adjective': 0, 'Adverb': 0}
    for word, tag in pos_tags:
        if tag.startswith('N'):
            pos_counts['Noun'] += 1
        elif tag.startswith('V'):
            pos_counts['Verb'] += 1
        elif tag.startswith('J'):
            pos_counts['Adjective'] += 1
        elif tag.startswith('R'):
            pos_counts['Adverb'] += 1
    return pos_counts

def constituency_parsing(text):
    parser = CoreNLPParser(url='http://localhost:9000')
    sentences = sent_tokenize(text)
    for sentence in sentences:
        trees = list(parser.parse(sentence.split()))
        print(f"Constituency parsing tree for: {sentence}")
        for tree in trees:
            print(tree)

def dependency_parsing(text):
    parser = CoreNLPParser(url='http://localhost:9000')
    sentences = sent_tokenize(text)
    for sentence in sentences:
        trees = list(parser.parse(sentence.split()))
        print(f"Dependency parsing tree for: {sentence}")
        for tree in trees:
            print(tree.tree())

def named_entity_recognition(text):
    sentences = sent_tokenize(text)
    all_entities = {'PERSON': 0, 'ORGANIZATION': 0, 'LOCATION': 0, 'PRODUCT': 0, 'DATE': 0}
    for sentence in sentences:
        words = word_tokenize(sentence)
        tagged_words = pos_tag(words)
        chunked_sentence = ne_chunk(tagged_words)
        for chunk in chunked_sentence:
            if hasattr(chunk, 'label'):
                entity_type = chunk.label()
                if entity_type in all_entities:
                    all_entities[entity_type] += 1
    return all_entities

if __name__ == "__main__":
    # Load the clean text from the CSV file
    with open('cleaned_g2_software_reviews.csv', 'r', encoding='utf-8') as file:
        text = file.read()

    # (1) Parts of Speech (POS) Tagging
    pos_counts = pos_tagging(text)
    print("Parts of Speech (POS) Tagging:")
    for pos, count in pos_counts.items():
        print(f"{pos}: {count}")

    # (2) Constituency Parsing
    print("\nConstituency Parsing:")
    constituency_parsing(text)

    # (3) Dependency Parsing
    print("\nDependency Parsing:")
    dependency_parsing(text)

    # (4) Named Entity Recognition
    entities = named_entity_recognition(text)
    print("\nNamed Entity Recognition:")
    for entity, count in entities.items():
        print(f"{entity}: {count}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


Parts of Speech (POS) Tagging:
Noun: 9005
Verb: 5000
Adjective: 5000
Adverb: 0

Constituency Parsing:


ConnectionError: HTTPConnectionPool(host='localhost', port=9000): Max retries exceeded with url: /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2Cpos%2Clemma%2Cssplit%2Cparse%22%2C+%22ssplit.eolonly%22%3A+%22true%22%7D (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x79ebca9354b0>: Failed to establish a new connection: [Errno 111] Connection refused'))

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [ ]:
# i learned a lot from the above code and its more challenge